<a href="https://colab.research.google.com/github/JanMeow/GNN_StackedRF_3D_Spatial_DeepLearning/blob/main/Chpater_2_GNN_Guess_the_GeometricType_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install pytorch
!pip install torch_geometric

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.5 MB/s eta 0:00:00


In [4]:
print(torch.__version__)

2.6.0+cu124


In [5]:
print(f"python {torch.version.cuda}")

python 12.4


In [6]:
import torch
torch.cuda.is_available()

False

In [3]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
Data(edge_index=[2, 4], x=[3, 1])

Data(x=[2], edge_index=[2])

#Transforming our graph into PYG graph

Although our data is in graph structure, it is different from the PyG format of graph we need for training.
So the first step would be to turn our graph into a PyG *graph*


1.   Turn node index into integers
2.   Use node.near property to connect the indexs



In [7]:
# Getting the data and data engineering functions
!git clone https://github.com/JanMeow/3d_graph_data_engineering.git
# Install the dependency
!mv 3d_graph_data_engineering data_engineering
!pip install numpy pandas ifcopenshell

Cloning into '3d_graph_data_engineering'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 91 (delta 52), reused 62 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 324.99 KiB | 13.00 MiB/s, done.
Resolving deltas: 100% (52/52), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.7 MB/s eta 0:00:00


In [8]:
import numpy as np
import pandas as pd
import ifcopenshell
import sys
sys.path.append('/content/data_engineering')
from data_engineering import collision as C, utils as U, compute_proxy as CP

In [9]:
from pathlib import Path
# lets use the test ifc_model
ifc_path = "/content/data_engineering/data/ifc/ML_train_data_0.ifc"

In [10]:
model = ifcopenshell.open(ifc_path)
root = model.by_type("IfcProject")[0]

In [11]:
from utils import  Graph
# Create a graph and establish BVH Tree=
graph = Graph.create(root)
graph.build_bvh()

Function ended, No more spatial child
Graph created


In [22]:
for node in graph.node_dict.values():
      if node.geom_info != None:
          node.near = [graph[guid] for guid in graph.bvh_query(node.geom_info["bbox"])
                        if guid != node.guid]

In [31]:
edge_index = []
for index, node in enumerate(graph.node_dict.values()):
  node.index = index
for node in graph.node_dict.values():
  for near_node in node.near:
    edge_index.append([node.index, near_node.index])
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

In [33]:
import torch
from torch_geometric.data import Data

x = torch.arange(len(graph))
data = Data(x=x, edge_index=edge_index)
data

Data(x=[312], edge_index=[2, 3414])